Enter all Greenhouse Gas (GHG) data from monitoring stormwater basins for Methane (CH4) and Nitrous Oxide (N2O).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
%matplotlib inline

In [2]:
#read in multiple csv viles and concatonate them all into 1 dataframe
path =r'C:\Users\Natalie\Box Sync\Basins - Campus\Collected Data\Greenhouse Gas\R Working Files\Processed Data' # use your path
allFiles = glob.glob(os.path.join(path, "GHGflux*.csv"))
data1 = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, na_values='NA')
    list_.append(df)
data1 = pd.concat(list_)

In [3]:
path =r'C:\Users\Natalie\Box Sync\Basins - Campus\Collected Data\Greenhouse Gas\R Working Files\Processed Data'
all_files = glob.glob(os.path.join(path, "GHGflux*.csv"))
data=pd.concat(pd.read_csv(f) for f in all_files)

In [4]:
#replace colum names
data.rename(columns={'flux.N2O': 'flux_N2O', 'flux.CH4': 'flux_CH4', 'flux.CO2':'flux_CO2'}, inplace=True)

data.head() #shows the dataframe we created

Unnamed: 0  ID       Date       Site   Location    VWC  Temp  Conductivity  \
0           1  E1  4/20/2016  Wet Basin      Basin  85.06   9.5          2.09   
1           2  E2  4/20/2016  Wet Basin      Basin  36.77  11.7          0.10   
2           3  E3  4/20/2016  Wet Basin      Basin  91.24   8.2          2.39   
3           4  E4  4/20/2016  Wet Basin  Reference  32.08  11.4          0.21   
4           5  E5  4/20/2016  Wet Basin  Reference  55.43  14.2          1.26   

   Soil.Porosity  Saturation  AirT     Rain  Precip   flux_N2O  r.squared.N2O  \
0           0.73        1.17   4.4  No Rain     NaN  -3.698828       0.003789   
1           0.68        0.54   4.4  No Rain     NaN  -0.354986       0.000036   
2           0.89        1.03   4.4  No Rain     NaN  21.720546       0.073681   
3           0.49        0.66   4.4  No Rain     NaN  20.625381       0.061650   
4           0.56        0.99   4.4  No Rain     NaN -35.320212       0.138446   

     flux_CH4  r.squared.CH4  r.squared.CO2       flux_CO2  
0  -22.757097       0.013714       0.608706  183595.128961  
1 -137.966454       0.647540       0.961278  190433.497043  
2   21.883248       0.250702       0.472880  -10212.923765  
3  -51.622236       0.346719       0.358094  126038.438749  
4   62.915548       0.293559       0.969641  225931.524478

In [5]:
data['Precip']=data['Precip'].fillna(0) #need to make sure we fill back in the origninal column with this new fill na 0
#set no rain events to precip=0 instead of NA

In [6]:
#import precipitation data
data_precip=pd.read_csv(r'C:\Users\Natalie\Box Sync\Basins - Campus\Collected Data\Greenhouse Gas\R Working Files\Sample Data\Basin_Precip.csv')
#needs the r infront of the string to make sure it does not accidientally think User is unicode text

##Remove data outliers greater than 4SD
#determine the mean methane flux value, then the standard deviation (SD). Remove values that are greater than or less than mean+4*SD

In [7]:
#how many nas exist now
data.flux_CH4.isnull().sum()

5

In [8]:
CH4_mean=data['flux_CH4'].mean()
CH4_sd=data['flux_CH4'].std()
CH4_4sd=CH4_mean+(4*CH4_sd)
data.loc[abs(data['flux_CH4'])>CH4_4sd, 'flux_CH4']

#df.loc[rows, columns] #in our case the rows are those that satisfy flux_CH4>4SD, and then it sets those values within the 
#Set value for rows matching callable condition, where rows match the column='shield', and the values in shield are >35
#df.loc[df['shield'] > 35] = 0

#df.loc[['viper', 'sidewinder'], ['shield']] = 50

2    42522.502544
0    60789.923953
0    34970.140311
0    51915.319631
0    52249.835596
Name: flux_CH4, dtype: float64

In [9]:
#replaces extreme values with NaN
data.loc[abs(data['flux_CH4'])>CH4_4sd, 'flux_CH4']=None
#selects columns were flux_CH4 < 4SD, and then in the column 'flux_CH4' it replaces these values with none. row, column

#data.flux_CH4[data.flux_CH4>CH4_4sd]=None #this one throws an error b/c it looks like it is chained and not done properly.

In [11]:
data.flux_CH4.isnull().sum() #counts up how many NAs

#data.loc[abs(data['flux_CH4'])>CH4_4sd, 'flux_CH4'] #no more outliers

Series([], Name: flux_CH4, dtype: float64)

Same thing for N2O outliers

In [447]:
N2O_mean=data['flux_N2O'].mean()
N2O_sd=data['flux_N2O'].std()
N2O_4sd=N2O_mean+(4*N2O_sd)
data.loc[abs(data['flux_N2O'])>N2O_4sd, 'flux_N2O']

2    5167.577487
0   -3300.613257
Name: flux_N2O, dtype: float64

In [448]:
data.flux_N2O.isnull().sum() #originally 5 NAs

5

In [449]:
#replaces extreme values with NaN
data.loc[abs(data['flux_N2O'])>N2O_4sd, 'flux_N2O']=None

data.flux_N2O.isnull().sum()#now there are 7, so we removed the 2 outliers

7

In [450]:
#remove soil saturation >1, as we cannot have greater than 100% saturated
data.loc[data['Saturation']>1, 'Saturation']=None

Get sum of 3-day rain prior to sampling event


In [451]:
data.shape[0] #total number of rows in the dataframe

340

In [452]:
pd.to_datetime('Mon Nov 02 20:37:10 GMT+00:00 2015')


Timestamp('2015-11-02 20:37:10')

In [12]:
data.Date=pd.to_datetime(data['Date'])
#xx.dtypes

In [454]:
data.dtypes
#date is an object variable instead of date

Unnamed: 0                int64
ID                       object
Date             datetime64[ns]
Site                     object
Location                 object
VWC                     float64
Temp                    float64
Conductivity            float64
Soil.Porosity           float64
Saturation              float64
AirT                    float64
Rain                     object
Precip                  float64
flux_N2O                float64
r.squared.N2O           float64
flux_CH4                float64
r.squared.CH4           float64
r.squared.CO2           float64
flux_CO2                float64
dtype: object

In [480]:
#data.Date.unique()

In [13]:
#make sure we can subtract 3 earlier days
from datetime import datetime, timedelta
data.Date.iloc[[1]]-timedelta(days=3)#this is correct, the original date was 4/20, now 3 days prior is 4/17

1   2016-04-17
Name: Date, dtype: datetime64[ns]

In [457]:
# data.loc[:,['Date']]
#same as above, select this column 
# data.Date
# data['Date']


In [14]:
data_precip.Date=pd.to_datetime(data_precip['Date'])
data_precip.head()

Date  Precip
0 2015-07-12     NaN
1 2015-07-13   0.508
2 2015-07-14  26.924
3 2015-07-15   0.508
4 2015-07-16   0.000

In [459]:
sample_date=data.Date.iloc[71]
days_prior=sample_date-timedelta(days=3)

In [460]:
mask=(data_precip['Date']>=days_prior) & (data_precip['Date']<sample_date)
data_precip.Precip[mask].sum()
#this works, but need it in loop
#(df['date'] > '2000-6-1') & (df['date'] <= '2000-6-10')

2.032

In [19]:
#create dummy dataframe
import random
sdate=range(1,10)
#start with dictionary
rawdata={'sdate': [1,4,6,12,15,16,20,24,28], 'precip': [0,0,1,2,3,5,5,0,1]}
dummy=pd.DataFrame(rawdata, columns=['sdate', 'precip'])

dummy.head()
#dummy.dtypes


sdate  precip
0      1       0
1      4       0
2      6       1
3     12       2
4     15       3

In [16]:
alldate=range(1,40)
precip_dummy=np.random.choice(10,39)#chooses 40 random numbers from 0-10
precip_dum=pd.DataFrame({'Date':alldate, 'precip_dummy':precip_dummy})
precip_dum.head()

Date  precip_dummy
0     1             8
1     2             7
2     3             2
3     4             7
4     5             8

In [17]:
sample_date=dummy.sdate.iloc[1]
days_prior=sample_date-3

mask=(precip_dum['Date']>=days_prior) & (precip_dum['Date']<sample_date)
#print(mask)
precip_dum.precip_dummy[mask].sum()

17

In [18]:
print(sample_date)
#precip_dum.loc[precip_dum['Date']==sample_date]

4


Date  precip_dummy
3     4             7

In [29]:
precip_out=[]
for x in dummy.sdate:
    sample_date=x
    days_prior=sample_date-3
    mask=(precip_dum['Date']>=days_prior) & (precip_dum['Date']<sample_date)
    precip_out.append(precip_dum.precip_dummy[mask].sum())

print(precip_out)    

[0, 17, 17, 11, 10, 17, 10, 20, 16]


In [30]:
print(sample_date)
print(days_prior)
print(precip_out)

28
25
[0, 17, 17, 11, 10, 17, 10, 20, 16]


In [48]:
#add new column to dataframe
#df['e'] = e
data['precip3']=None
data.head()

Unnamed: 0  ID       Date       Site   Location    VWC  Temp  Conductivity  \
0           1  E1 2016-04-20  Wet Basin      Basin  85.06   9.5          2.09   
1           2  E2 2016-04-20  Wet Basin      Basin  36.77  11.7          0.10   
2           3  E3 2016-04-20  Wet Basin      Basin  91.24   8.2          2.39   
3           4  E4 2016-04-20  Wet Basin  Reference  32.08  11.4          0.21   
4           5  E5 2016-04-20  Wet Basin  Reference  55.43  14.2          1.26   

   Soil.Porosity  Saturation  AirT     Rain  Precip   flux_N2O  r.squared.N2O  \
0           0.73        1.17   4.4  No Rain     0.0  -3.698828       0.003789   
1           0.68        0.54   4.4  No Rain     0.0  -0.354986       0.000036   
2           0.89        1.03   4.4  No Rain     0.0  21.720546       0.073681   
3           0.49        0.66   4.4  No Rain     0.0  20.625381       0.061650   
4           0.56        0.99   4.4  No Rain     0.0 -35.320212       0.138446   

     flux_CH4  r.squared.CH4  r.squared.CO2       flux_CO2 precip3  
0  -22.757097       0.013714       0.608706  183595.128961    None  
1 -137.966454       0.647540       0.961278  190433.497043    None  
2   21.883248       0.250702       0.472880  -10212.923765    None  
3  -51.622236       0.346719       0.358094  126038.438749    None  
4   62.915548       0.293559       0.969641  225931.524478    None

In [55]:
precip3=[]
j=[]

for x in data.Date:
    sample_date=x
    days_prior=sample_date-timedelta(days=3)
    mask=(data_precip['Date']>=days_prior) & (data_precip['Date']<sample_date)
    precip3.append(data_precip.Precip[mask].sum())
    j+=[data_precip.Precip[mask].sum()] #same thing as above, means add to next 1 in j
      
data['precip3']=precip3

In [54]:
data.head()

Unnamed: 0  ID       Date       Site   Location    VWC  Temp  Conductivity  \
0           1  E1 2016-04-20  Wet Basin      Basin  85.06   9.5          2.09   
1           2  E2 2016-04-20  Wet Basin      Basin  36.77  11.7          0.10   
2           3  E3 2016-04-20  Wet Basin      Basin  91.24   8.2          2.39   
3           4  E4 2016-04-20  Wet Basin  Reference  32.08  11.4          0.21   
4           5  E5 2016-04-20  Wet Basin  Reference  55.43  14.2          1.26   

   Soil.Porosity  Saturation  AirT     Rain  Precip   flux_N2O  r.squared.N2O  \
0           0.73        1.17   4.4  No Rain     0.0  -3.698828       0.003789   
1           0.68        0.54   4.4  No Rain     0.0  -0.354986       0.000036   
2           0.89        1.03   4.4  No Rain     0.0  21.720546       0.073681   
3           0.49        0.66   4.4  No Rain     0.0  20.625381       0.061650   
4           0.56        0.99   4.4  No Rain     0.0 -35.320212       0.138446   

     flux_CH4  r.squared.CH4  r.squared.CO2       flux_CO2  precip3  
0  -22.757097       0.013714       0.608706  183595.128961      0.0  
1 -137.966454       0.647540       0.961278  190433.497043      0.0  
2   21.883248       0.250702       0.472880  -10212.923765      0.0  
3  -51.622236       0.346719       0.358094  126038.438749      0.0  
4   62.915548       0.293559       0.969641  225931.524478      0.0

In [491]:
# data.Date[1] #returns all unique dates
#data.iloc[0]['Date']

Average by timepoint for each Site, N2O anc CH4 fluxes are the average of 3 sample points (3 within each basin) at each sample day

In [75]:
#Average by Site, Location, and Date

data_time=data.groupby(by=['Site', 'Location', 'Date']).mean() #default does by columns, so takes the mean of all data within tho=se columns
data_time['Rain']=np.where(data_time['Precip']>0, 'YesRain', 'NoRain')
data_time.head()


Unnamed: 0        VWC       Temp  Conductivity  \
Site      Location Date                                                         
Dry Basin Basin    2016-04-20         7.0  32.880000  14.200000      0.263333   
                   2016-04-27         7.0  38.335667  14.600000      0.266667   
                   2016-05-11         7.0  35.510000  16.700000      0.240000   
                   2016-06-02         7.0  31.850000  23.833333      0.266667   
                   2016-06-20         7.0  23.056333  28.333333      0.040000   

                               Soil.Porosity  Saturation       AirT  Precip  \
Site      Location Date                                                       
Dry Basin Basin    2016-04-20        0.59000    0.556667   4.400000   0.000   
                   2016-04-27        0.58937    0.651611   5.000000   8.128   
                   2016-05-11        0.59000    0.603333  12.222222   0.000   
                   2016-06-02        0.59000    0.540000  19.722222   3.050   
                   2016-06-20        0.58937    0.391169  22.500000   0.000   

                                flux_N2O  r.squared.N2O    flux_CH4  \
Site      Location Date                                               
Dry Basin Basin    2016-04-20  15.301894       0.422235  119.952659   
                   2016-04-27  46.593877       0.526030   35.442345   
                   2016-05-11 -20.608212       0.148402   33.570299   
                   2016-06-02  35.070078       0.412247 -337.045412   
                   2016-06-20  -4.409587       0.119157   -9.254231   

                               r.squared.CH4  r.squared.CO2       flux_CO2  \
Site      Location Date                                                      
Dry Basin Basin    2016-04-20       0.385080       0.614934   57531.891553   
                   2016-04-27       0.412516       0.557980  124193.009623   
                   2016-05-11       0.606741       0.715517  289027.235717   
                   2016-06-02       0.520968       0.392590  164310.741400   
                   2016-06-20       0.529375       0.635701  149144.340207   

                               precip3     Rain  
Site      Location Date                          
Dry Basin Basin    2016-04-20    0.000   NoRain  
                   2016-04-27   10.922  YesRain  
                   2016-05-11    0.000   NoRain  
                   2016-06-02    0.000  YesRain  
                   2016-06-20    0.000   NoRain

In [116]:
#Overall averages at each site, over the entire time period
data_avg=data.groupby(by=['Site', 'Location']).mean()


Unnamed: 0        VWC       Temp  Conductivity  \
Site      Location                                                    
Dry Basin Basin        6.529412  31.594588  21.948039      0.207941   
          Reference    9.044118  26.610602  21.838276      0.104200   
Wet Basin Basin        2.000000  59.209239  19.064356      1.075294   
          Reference    4.514706  36.390771  20.328788      0.443182   

                     Soil.Porosity  Saturation       AirT    Precip  \
Site      Location                                                    
Dry Basin Basin           0.587855    0.536780  18.364804  5.352059   
          Reference       0.524056    0.509176  18.364804  5.352059   
Wet Basin Basin           0.758604    0.765266  18.364804  5.352059   
          Reference       0.524672    0.689601  18.364804  5.352059   

                      flux_N2O  r.squared.N2O     flux_CH4  r.squared.CH4  \
Site      Location                                                          
Dry Basin Basin      14.477479       0.444328   -53.030841       0.380346   
          Reference   7.140621       0.347164    26.694429       0.397037   
Wet Basin Basin      25.185981       0.418293  4318.903437       0.675953   
          Reference  -3.329264       0.346685   115.356477       0.384468   

                     r.squared.CO2       flux_CO2    precip3  
Site      Location                                            
Dry Basin Basin           0.815182  573160.090887  10.914529  
          Reference       0.859385  570403.972197  10.914529  
Wet Basin Basin           0.837456  315566.785286  10.914529  
          Reference       0.792544  626131.314976  10.914529

Average flux at teach timepoint, with std, count etc. Do it for N2O and CH4 then concatonate them together

In [144]:
#syntax works, but leaves index
n2o_agg=data.groupby(['Site', 'Location', 'Date'], as_index=False)['flux_N2O'].agg(['count','mean', 'std', 'max', 'min'], as_index=False)
 #indexes are still there instead of filled in


count       mean        std        max  \
Site      Location Date                                                 
Dry Basin Basin    2016-04-20      3  15.301894  52.798109  69.103772   
                   2016-04-27      3  46.593877  13.219106  59.131550   
                   2016-05-11      3 -20.608212  11.255629 -11.158864   
                   2016-06-02      3  35.070078  29.780153  63.171969   
                   2016-06-20      3  -4.409587   8.524318   5.159191   

                                     min  
Site      Location Date                   
Dry Basin Basin    2016-04-20 -36.431660  
                   2016-04-27  32.785158  
                   2016-05-11 -33.060858  
                   2016-06-02   3.856082  
                   2016-06-20 -11.192190

In [152]:
list(n2o_agg)

['count', 'mean', 'std', 'max', 'min']

In [145]:
N2O_agg=data.groupby(['Site', 'Location', 'Date'], as_index=False).agg({'flux_N2O': ['count','mean', 'std', 'max', 'min']})

N2O_agg.head()

Site Location       Date flux_N2O                                   \
                                    count       mean        std        max   
0  Dry Basin    Basin 2016-04-20        3  15.301894  52.798109  69.103772   
1  Dry Basin    Basin 2016-04-27        3  46.593877  13.219106  59.131550   
2  Dry Basin    Basin 2016-05-11        3 -20.608212  11.255629 -11.158864   
3  Dry Basin    Basin 2016-06-02        3  35.070078  29.780153  63.171969   
4  Dry Basin    Basin 2016-06-20        3  -4.409587   8.524318   5.159191   

              
         min  
0 -36.431660  
1  32.785158  
2 -33.060858  
3   3.856082  
4 -11.192190

In [146]:
list(N2O_agg) #need to rename columns to be list and not touple

[('Site', ''),
 ('Location', ''),
 ('Date', ''),
 ('flux_N2O', 'count'),
 ('flux_N2O', 'mean'),
 ('flux_N2O', 'std'),
 ('flux_N2O', 'max'),
 ('flux_N2O', 'min')]

In [151]:
N2O_agg.columns = ['Site', 'Location','Date','count','mean','std','max','min']
N2O_agg['type']='N2O_flux'
list(N2O_agg)

['Site', 'Location', 'Date', 'count', 'mean', 'std', 'max', 'min', 'type']

In [153]:
#same thing for CH4
CH4_agg=data.groupby(['Site', 'Location', 'Date'], as_index=False).agg({'flux_CH4': ['count','mean', 'std', 'max', 'min']})
CH4_agg.columns = ['Site', 'Location','Date','count','mean','std','max','min']
CH4_agg['type']='CH4_flux'

In [157]:
data_Avg_date_Site=pd.concat([N2O_agg, CH4_agg])
data_Avg_date_Site.head()

Site Location       Date  count       mean        std        max  \
0  Dry Basin    Basin 2016-04-20      3  15.301894  52.798109  69.103772   
1  Dry Basin    Basin 2016-04-27      3  46.593877  13.219106  59.131550   
2  Dry Basin    Basin 2016-05-11      3 -20.608212  11.255629 -11.158864   
3  Dry Basin    Basin 2016-06-02      3  35.070078  29.780153  63.171969   
4  Dry Basin    Basin 2016-06-20      3  -4.409587   8.524318   5.159191   

         min      type  
0 -36.431660  N2O_flux  
1  32.785158  N2O_flux  
2 -33.060858  N2O_flux  
3   3.856082  N2O_flux  
4 -11.192190  N2O_flux

In [130]:
data['Rain']=np.where(data['Precip']>0, 'YesRain', 'NoRain')
#if no rain make new column

Save file to csv

In [161]:
np.savetxt('C:\Users\Natalie\Dropbox\Python Training 2018\GHG Recreate\ProcessedData\data_Avg_date_Site.csv', data_Avg_date_Site)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-161-70bad92e4c2c>, line 1)

In [165]:
data_Avg_date_Site.to_csv('ProcessedData\data_Avg_date_Site.csv')
data.to_csv('ProcessedData\data.csv')